# Process Data

In [ ]:
import pandas as pd

data=pd.read_csv("df_grouped_with_summaries_Prompt 3.csv")
data.head()
# data.columns

In [ ]:
from sklearn.preprocessing import StandardScaler

# Assuming 'age' is the column name containing age data
if 'age' in data.columns:
  scaler = StandardScaler()
  data['age_standardized'] = scaler.fit_transform(data[['age']])
  print(data[['age', 'age_standardized']].head())
else:
  print("Error: 'age' column not found in the DataFrame.")


In [3]:
# data=data.iloc[:100,:]
# data = data.fillna('None')
# data

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'data' DataFrame is already loaded and processed as in the previous code

# Group data by 'case_status'
grouped_data = data.groupby('case_status')

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Iterate through the groups
for case_status, group_data in grouped_data:
  # Select the appropriate subplot
  ax = axes[case_status]

  # Plot the distribution of 'age' for the current group
  group_data['age'].hist(ax=ax, bins=20)

  # Set the title and labels for the subplot
  ax.set_title(f"Age Distribution for case_status = {case_status}")
  ax.set_xlabel("Age")
  ax.set_ylabel("Frequency")

# Display the plots
plt.tight_layout()
plt.show()


In [ ]:
# prompt: check if there's null values in gemini_summary

# Check for null values in the 'gemini_summary' column
null_values = data['gemini_summary'].isnull().sum()
print(f"Number of null values in 'gemini_summary': {null_values}")

# Check for empty strings in the 'gemini_summary' column
empty_strings = (data['gemini_summary'] == '').sum()
print(f"Number of empty strings in 'gemini_summary': {empty_strings}")

# Check for "None" or "nan" strings in 'gemini_summary'
none_strings = (data['gemini_summary'] == 'None').sum()
nan_strings = (data['gemini_summary'].str.lower() == 'nan').sum()
print(f"Number of 'None' strings in 'gemini_summary': {none_strings}")
print(f"Number of 'nan' strings in 'gemini_summary': {nan_strings}")


# Build Model

In [6]:
# !pip install transformers
# !pip install sentence-transformers

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load ClinicalBERT and tokenizer
model_name = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)
model.eval()

# Define feature and target columns
feature_cols = ['age_standardized', 'gender', 'insurance_group', 'language_group', 'race_group1','Stroke_History', 'Myocardial_Infarction', 'Peripheral_Vascular_Disease',
                'Cerebrovascular_Disease', 'Diabetes_Mellitus', 'Cancer']
target_col = 'case_status'

# Split dataset first
X_train_df, X_test_df, y_train, y_test = train_test_split(
    data, data[target_col], test_size=0.2, random_state=42
)

from tqdm import tqdm

# Embedding function with GPU support + tqdm
def generate_embeddings(texts):
    embeddings = []
    for text in tqdm(texts, desc="Generating Embeddings"):
        if pd.isna(text):
            text = ""
        else:
            text = str(text)
        inputs = tokenizer(text, truncation=True, padding='max_length', max_length=512, return_tensors='pt').to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
        embeddings.append(cls_embedding)
    return np.array(embeddings)

# Generate embeddings
X_train_embeddings = generate_embeddings(X_train_df['gemini_summary'].tolist())
X_test_embeddings = generate_embeddings(X_test_df['gemini_summary'].tolist())


In [ ]:
X_train_embeddings.shape

# Remove low variance embedding features

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Scale the embeddings
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_embeddings)
X_test_scaled = scaler.transform(X_test_embeddings)

# Apply PCA
pca = PCA()
X_train_pca = pca.fit_transform(X_train_scaled)

# Explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_

# Find optimal k (number of principal components)
cumulative_variance = np.cumsum(explained_variance_ratio)
k = np.argmax(cumulative_variance >= 0.80) + 1  # Example: keep components explaining 80% variance

print(f"Optimal number of components (k): {k}")

# Apply PCA with optimal k
pca = PCA(n_components=k)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Now X_train_pca and X_test_pca contain the top k principal components
print(f"Shape of X_train_pca: {X_train_pca.shape}")
print(f"Shape of X_test_pca: {X_test_pca.shape}")


In [10]:
# from sklearn.feature_selection import VarianceThreshold
# selector = VarianceThreshold(threshold=1e-4)
# X_train_embeddings = selector.fit_transform(X_train_embeddings)
# X_test_embeddings = selector.transform(X_test_embeddings)


#### LightGBM with RandomizedSearchCV for hyperparameter tuning

| **Benefit**                        | **LightGBM**                              | **Logistic Regression**                   |
|------------------------------------|-------------------------------------------|--------------------------------------------|
| **Nonlinear relationships**        | ✅ Captures complex, nonlinear patterns    | ❌ Assumes linearity between features & log-odds |
| **Feature interactions**           | ✅ Automatically models feature interactions | ❌ Cannot model interactions unless manually created |
| **Handling many features**         | ✅ Efficient with large/sparse feature sets | ⚠️ Needs strong regularization to avoid overfitting |
| **Automatic feature selection**    | ✅ Ignores uninformative features (via gain) | ❌ All features contribute unless explicitly regularized |
| **Robust to feature scaling**      | ✅ No scaling needed                       | ❌ Requires standardized features           |
| **Tree-based regularization**      | ✅ via subsample, colsample, L1/L2         | ⚠️ Relies on proper penalty tuning (e.g. `C`, `l1_ratio`) |
| **Handles missing values**         | ✅ Built-in support                        | ❌ Requires imputation                      |
| **Performance (usually)**          | ✅ Often higher accuracy/F1 on complex datasets | ⚠️ Weaker when relationships are nonlinear |

In [11]:
# === Step 1: Define categorical columns ===
categorical_cols = ['gender', 'insurance_group', 'language_group', 'race_group1',
                    'Stroke_History', 'Myocardial_Infarction', 'Peripheral_Vascular_Disease',
                    'Cerebrovascular_Disease', 'Diabetes_Mellitus', 'Cancer']

# === Step 2: Ensure categorical columns are dtype 'category' ===
for col in categorical_cols:
    X_train_df[col] = X_train_df[col].astype('category')
    X_test_df[col] = X_test_df[col].astype('category')

# === Step 3: Build structured matrix with categorical + numerical ===
structured_cols = categorical_cols + ['age_standardized']
X_train_structured = X_train_df[structured_cols]
X_test_structured = X_test_df[structured_cols]

# === Step 4: Combine embeddings with structured features ===
X_train_full = pd.concat([pd.DataFrame(X_train_pca), X_train_structured.reset_index(drop=True)], axis=1)
X_test_full = pd.concat([pd.DataFrame(X_test_pca), X_test_structured.reset_index(drop=True)], axis=1)


In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Step 1: Define the reduced parameter grid
param_grid = {
    'n_estimators': [50, 100],  # Add more for longer training if early stopping is used
    'learning_rate': [0.01, 0.05],  # Try slower learning for smoother convergence
    'max_depth': [4, 6],                # Shallower trees to reduce complexity
    'bagging_fraction': [0.5, 0.6],   # More aggressive row subsampling
    'bagging_freq': [1, 3],             # Try more frequent bagging
    'colsample_bytree': [0.5, 0.6],   # Try smaller feature subsampling
    'reg_alpha': [0.1, 0.8],          # L1 regularization
    'reg_lambda': [ 0.1, 0.8]          # L2 regularization
}


# Step 2: Set up GridSearchCV
grid = GridSearchCV(
    LGBMClassifier(
        random_state=42,
        n_jobs=-1,
        verbosity=-1
    ),
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    verbose=1
)

# Step 3: Fit the model
grid.fit(X_train_full, y_train)

# Step 4: Evaluate best model
print("Best params:", grid.best_params_)
y_pred = grid.best_estimator_.predict(X_test_full)
print(classification_report(y_test, y_pred))

In [13]:
# Predict probabilities and labels using tuned model and updated features
clf=grid.best_estimator_
train_probs = clf.predict_proba(X_train_full)
test_probs = clf.predict_proba(X_test_full)

# Add predictions to the original dataframes
X_train_df['predicted_probability'] = np.max(train_probs, axis=1)
X_test_df['predicted_probability'] = np.max(test_probs, axis=1)

X_train_df['prediction'] = clf.predict(X_train_full)
X_test_df['prediction'] = clf.predict(X_test_full)

In [14]:
# Combine train and test for consistent one-hot encoding
X_train_df['split'] = 'train'
X_test_df['split'] = 'test'

final_df = pd.concat([X_train_df, X_test_df])
final_df.to_csv('final_df.csv', index=False)

In [ ]:
final_df.head()

In [ ]:
from lightgbm import plot_importance

plot_importance(clf, max_num_features=40, importance_type='split')  # or 'gain'
plt.title('LightGBM Feature Importance')
plt.show()


# Test Performance

⚖️ Why These Metrics?

In a classification task (e.g., predicting case_status), fairness means the model's decisions should not unfairly disadvantage people from certain groups (like gender, race, age, etc.).

To assess this, we compare how well or how consistently the model performs across different subgroups. Each of the three metrics measures a specific kind of fairness.

🔹 1. Equal Opportunity (TPR)

The True Positive Rate (TP / [TP + FN]) should be equal across groups.

Why it matters:

This ensures that qualified individuals (i.e., people who truly belong to the positive class) are equally likely to be correctly identified, no matter their demographic. It aims for equal performance for both positive and negative predictions across groups, ensuring that neither group benefits or suffers disproportionately from the model's decisions.

Example: In healthcare, all patients who actually need treatment (true positives) should have an equal chance of being correctly identified — regardless of race or gender.

Bias implication:

If Group A has TPR = 0.9 and Group B has TPR = 0.6 → the model is less likely to help Group B, even if they qualify. That’s discriminatory.

🔹 2. Predictive Parity (Precision)
The Precision (TP / [TP + FP]) should be equal across groups.

Why it matters:

This ensures that those who are predicted to be positive actually are, at equal rates across groups.

In practical terms, it means the model is equally trustworthy in its positive predictions for all groups.

Bias implication:

If Group A has 80% precision and Group B has 50% → Group B suffers more false alarms or unjustified interventions, which is unfair.

🔹 3. Equalized Odds (TPR & FPR)

Both TPR and FPR (False Positive Rate = FP / [FP + TN]) should be equal across groups.

Why it matters:

This is the most comprehensive fairness metric.

It ensures the model:

✅ correctly identifies positives at the same rate (Equal Opportunity)

❌ doesn’t falsely accuse negatives at different rates (False alarms)

Bias implication:

If Group A has FPR = 0.1 and Group B has FPR = 0.3 → Group B receives more false accusations, which can be harmful or costly (e.g., wrongful arrests, misdiagnoses).

🧠 Summary Table

Metric	Measures	Prevents Bias Where...

Equal Opportunity	TPR	Qualified individuals are denied more often in some groups

Predictive Parity	Precision	Some groups receive more incorrect positive predictions

Equalized Odds	TPR & FPR	Some groups are both over- or under-predicted unfairly


#### Overall

In [17]:
final_df=pd.read_csv("final_df.csv")

In [ ]:
from sklearn.metrics import confusion_matrix

# Use only test set
test_data = final_df[final_df['split'] == 'train']

# Get ground truth and predictions
y_true = test_data['case_status']
y_pred = test_data['prediction']

# Compute confusion matrix
try:
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
except ValueError:
    # Handle edge cases where only one class is present
    tn = fp = fn = tp = 0
    for actual, pred in zip(y_true, y_pred):
        if actual == 1 and pred == 1:
            tp += 1
        elif actual == 1 and pred == 0:
            fn += 1
        elif actual == 0 and pred == 1:
            fp += 1
        elif actual == 0 and pred == 0:
            tn += 1

# Compute fairness-related metrics
tpr = tp / (tp + fn) if (tp + fn) > 0 else 0.0  # Sensitivity / Equal Opportunity
fpr = fp / (fp + tn) if (fp + tn) > 0 else 0.0  # False Positive Rate / Equalized Odds
precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0  # Precision / Predictive Parity
sample_size = len(test_data)

# Print results
print("📊 Overall Model Performance on Train Set")
print("-" * 40)
print(f"TPR:       {tpr:.4f}")
print(f"FPR:       {fpr:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Sample Size:  {sample_size}")

In [ ]:
from sklearn.metrics import confusion_matrix

# Use only test set
test_data = final_df[final_df['split'] == 'test']

# Get ground truth and predictions
y_true = test_data['case_status']
y_pred = test_data['prediction']

# Compute confusion matrix
try:
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
except ValueError:
    # Handle edge cases where only one class is present
    tn = fp = fn = tp = 0
    for actual, pred in zip(y_true, y_pred):
        if actual == 1 and pred == 1:
            tp += 1
        elif actual == 1 and pred == 0:
            fn += 1
        elif actual == 0 and pred == 1:
            fp += 1
        elif actual == 0 and pred == 0:
            tn += 1

# Compute fairness-related metrics
tpr = tp / (tp + fn) if (tp + fn) > 0 else 0.0  # Sensitivity / Equal Opportunity
fpr = fp / (fp + tn) if (fp + tn) > 0 else 0.0  # False Positive Rate / Equalized Odds
precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0  # Precision / Predictive Parity
sample_size = len(test_data)

# Print results
print("📊 Overall Model Performance on Test Set")
print("-" * 40)
print(f"TPR:       {tpr:.4f}")
print(f"FPR:       {fpr:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Sample Size:  {sample_size}")

# Bias Validation

## Sample performance

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from tabulate import tabulate

# === Age binning ===
def bin_age(df):
    bin_edges = [0, 70, 80, 90, float('inf')]
    labels = ['<70', '70-80', '80-90', '>90']
    df['age_group'] = pd.cut(df['age'], bins=bin_edges, labels=labels, right=False)
    return df

# === Bootstrap metrics: TPR, FPR, Precision only ===
def bootstrap_metrics(y_true, y_pred, n_iterations=100):
    stats = {'TPR': [], 'FPR': [], 'Precision': []}
    for _ in range(n_iterations):
        try:
            sample = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred}).sample(frac=1.0, replace=True)
            yt = sample['y_true']
            yp = sample['y_pred']
            tn, fp, fn, tp = confusion_matrix(yt, yp, labels=[0, 1]).ravel()
            tpr = tp / (tp + fn) if (tp + fn) > 0 else np.nan
            fpr = fp / (fp + tn) if (fp + tn) > 0 else np.nan
            precision = tp / (tp + fp) if (tp + fp) > 0 else np.nan
            stats['TPR'].append(tpr)
            stats['FPR'].append(fpr)
            stats['Precision'].append(precision)
        except:
            continue

    return {
        'TPR': (np.nanpercentile(stats['TPR'], 2.5), np.nanpercentile(stats['TPR'], 97.5)),
        'FPR': (np.nanpercentile(stats['FPR'], 2.5), np.nanpercentile(stats['FPR'], 97.5)),
        'Precision': (np.nanpercentile(stats['Precision'], 2.5), np.nanpercentile(stats['Precision'], 97.5)),
    }

# === Apply binning to age ===
final_df = bin_age(final_df)

# === Split data ===
test_data = final_df[final_df['split'] == 'test']
train_data = final_df[final_df['split'] == 'train']

# === Demographic groups ===
demographic_groups = ['age_group', 'gender', 'insurance_group', 'language_group',
                      'race_group1', 'race_group2', 'race_group3', 'race_group4']

# === Run analysis for Train and Test ===
for data_split_name, data in zip(['Train', 'Test'], [train_data, test_data]):
    records = []

    for group_col in demographic_groups:
        if group_col not in data.columns:
            continue

        group_values = data[group_col].dropna().unique()

        for value in group_values:
            subset = data[data[group_col] == value]
            if len(subset) < 10:
                continue  # skip small groups

            y_true = subset['case_status']
            y_pred = subset['prediction']
            ci = bootstrap_metrics(y_true, y_pred)

            records.append({
                'Group': group_col,
                'Subgroup': value,
                'Equal Opportunity (TPR)': f"{ci['TPR'][0]:.3f}–{ci['TPR'][1]:.3f}",
                'False Positive Rate (FPR)': f"{ci['FPR'][0]:.3f}–{ci['FPR'][1]:.3f}",
                'Precision (Predictive Parity)': f"{ci['Precision'][0]:.3f}–{ci['Precision'][1]:.3f}",
                'Sample Size': len(subset)
            })

    df_metrics = pd.DataFrame(records)
    print(f"\n=== {data_split_name} Fairness Metrics Per Subgroup ===")
    print(tabulate(df_metrics, headers="keys", tablefmt="fancy_grid", showindex=False))


## Check Equalized Opportunity Violations -  pairwise

`proportions_ztest`: Compares proportions between two independent groups to see if they're different

In [ ]:
from statsmodels.stats.proportion import proportions_ztest
from sklearn.metrics import confusion_matrix
import pandas as pd

def compute_tpr_components(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    return tp, fn  # TPR = TP / (TP + FN)

def test_tpr_difference(df, group_col, y_true_col='case_status', y_pred_col='prediction'):
    values = df[group_col].dropna().unique()
    results = []

    for i in range(len(values)):
        for j in range(i+1, len(values)):
            g1, g2 = values[i], values[j]
            df1 = df[df[group_col] == g1]
            df2 = df[df[group_col] == g2]

            if len(df1) < 10 or len(df2) < 10:
                continue  # skip small groups

            try:
                tp1, fn1 = compute_tpr_components(df1[y_true_col], df1[y_pred_col])
                tp2, fn2 = compute_tpr_components(df2[y_true_col], df2[y_pred_col])
            except:
                continue

            count = [tp1, tp2]
            nobs = [tp1 + fn1, tp2 + fn2]

            if min(nobs) == 0:
                continue  # skip invalid

            stat, pval = proportions_ztest(count, nobs)
            if pval < 0.05:
                results.append({
                    'Demographic': group_col,
                    'Group 1': g1,
                    'Group 2': g2,
                    'TPR 1': f"{tp1 / nobs[0]:.3f}",
                    'TPR 2': f"{tp2 / nobs[1]:.3f}",
                    'p-value': round(pval, 4)
                })

    return pd.DataFrame(results)

# === Run for each demographic column ===
demographic_groups = ['age_group', 'gender', 'insurance_group', 'language_group',
                      'race_group1', 'race_group2', 'race_group3', 'race_group4']

for group_col in demographic_groups:
    print(f"\n=== TPR differences in {group_col} ===")
    df_tpr_diff = test_tpr_difference(final_df[final_df['split']=="test"], group_col)
    if df_tpr_diff.empty:
        print("No significant TPR differences.")
    else:
        print(df_tpr_diff)


## Check Equalized Opportunity Violations - group

`chi2_contingency`

In [ ]:
from sklearn.metrics import confusion_matrix
from scipy.stats import chi2_contingency
import pandas as pd

def chi2_test_tpr(df, group_col, y_true_col='case_status', y_pred_col='prediction'):
    table = []
    labels = []

    for val in df[group_col].dropna().unique():
        subset = df[df[group_col] == val]
        if len(subset) < 10:
            continue

        try:
            tn, fp, fn, tp = confusion_matrix(subset[y_true_col], subset[y_pred_col], labels=[0,1]).ravel()
        except:
            continue

        table.append([tp, fn])  # TPR = TP / (TP + FN)
        labels.append(val)

    if len(table) < 2:
        return None, None, None

    stat, pval, _, expected = chi2_contingency(table)

    return pd.DataFrame(table, index=labels, columns=['TP', 'FN']), pval, stat

# Example usage:
for group in demographic_groups:
    print(f"\n=== TPR difference test across {group} ===")
    contingency_df, pval, stat = chi2_test_tpr(final_df[final_df['split']=="test"], group)

    if contingency_df is None:
        print("Not enough valid groups.")
    else:
        print(contingency_df)
        print(f"Chi-squared p-value: {pval:.4f}")
        if pval < 0.05:
            print("✅ Significant TPR difference across subgroups.")
        else:
            print("❌ No significant TPR difference across subgroups.")


## Check Predictive Parity Violations -  Pairwise

In [ ]:
from statsmodels.stats.proportion import proportions_ztest
from sklearn.metrics import confusion_matrix
import pandas as pd

def compute_precision_components(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    return tp, fp  # Precision = TP / (TP + FP)

def test_precision_difference(df, group_col, y_true_col='case_status', y_pred_col='prediction'):
    values = df[group_col].dropna().unique()
    results = []

    for i in range(len(values)):
        for j in range(i+1, len(values)):
            g1, g2 = values[i], values[j]
            df1 = df[df[group_col] == g1]
            df2 = df[df[group_col] == g2]

            if len(df1) < 10 or len(df2) < 10:
                continue  # skip small groups

            try:
                tp1, fp1 = compute_precision_components(df1[y_true_col], df1[y_pred_col])
                tp2, fp2 = compute_precision_components(df2[y_true_col], df2[y_pred_col])
            except:
                continue

            count = [tp1, tp2]
            nobs = [tp1 + fp1, tp2 + fp2]

            if min(nobs) == 0:
                continue  # skip invalid

            stat, pval = proportions_ztest(count, nobs)
            if pval < 0.05:
                results.append({
                    'Demographic': group_col,
                    'Group 1': g1,
                    'Group 2': g2,
                    'Precision 1': f"{tp1 / nobs[0]:.3f}",
                    'Precision 2': f"{tp2 / nobs[1]:.3f}",
                    'p-value': round(pval, 4)
                })

    return pd.DataFrame(results)
for group_col in demographic_groups:
    print(f"\n=== Precision differences in {group_col} ===")
    df_precision_diff = test_precision_difference(final_df[final_df['split']=="test"], group_col)
    if df_precision_diff.empty:
        print("No significant Precision differences.")
    else:
        print(df_precision_diff)

## Check Predictive Parity Violations -  group

In [ ]:
from sklearn.metrics import confusion_matrix
from scipy.stats import chi2_contingency
import pandas as pd

def chi2_test_precision(df, group_col, y_true_col='case_status', y_pred_col='prediction'):
    table = []
    labels = []

    for val in df[group_col].dropna().unique():
        subset = df[df[group_col] == val]
        if len(subset) < 10:
            continue

        try:
            tn, fp, fn, tp = confusion_matrix(subset[y_true_col], subset[y_pred_col], labels=[0, 1]).ravel()
        except:
            continue

        table.append([tp, fp])  # Precision = TP / (TP + FP)
        labels.append(val)

    if len(table) < 2:
        return None, None, None

    stat, pval, _, expected = chi2_contingency(table)

    return pd.DataFrame(table, index=labels, columns=['TP', 'FP']), pval, stat

for group in demographic_groups:
    print(f"\n=== Precision difference test across {group} ===")
    contingency_df, pval, stat = chi2_test_precision(final_df[final_df['split']=="test"], group)

    if contingency_df is None:
        print("Not enough valid groups.")
    else:
        print(contingency_df)
        print(f"Chi-squared p-value: {pval:.4f}")
        if pval < 0.05:
            print("✅ Significant Precision difference across subgroups.")
        else:
            print("❌ No significant Precision difference across subgroups.")

## Check Equalized odds Violations -  pairwise


In [ ]:
pip install fairlearn

In [ ]:
import pandas as pd
import numpy as np
from fairlearn.metrics import equalized_odds_difference
from itertools import combinations
from tabulate import tabulate

# === Age binning ===
def bin_age(df):
    bin_edges = [0, 70, 80, 90, float('inf')]
    labels = ['<70', '70-80', '80-90', '>90']
    df['age_group'] = pd.cut(df['age'], bins=bin_edges, labels=labels, right=False)
    return df

# === Bootstrap EOD for two subgroups only ===
def bootstrap_pairwise_eod(y_true, y_pred, sensitive_features, n_iterations=100):
    eods = []
    data = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred, 'sensitive': sensitive_features})

    for _ in range(n_iterations):
        sample = data.sample(frac=1.0, replace=True)
        try:
            eod = equalized_odds_difference(sample['y_true'], sample['y_pred'], sensitive_features=sample['sensitive'])
            eods.append(eod)
        except:
            continue

    if len(eods) == 0:
        return (np.nan, np.nan)

    return (
        np.nanpercentile(eods, 2.5),
        np.nanpercentile(eods, 97.5)
    )

# === Apply age binning ===
final_df = bin_age(final_df)

# === Filter to test set only ===
test_data = final_df[final_df['split'] == 'test']

# === Demographic groups ===
demographic_groups = ['age_group', 'gender', 'insurance_group', 'language_group',
                      'race_group1', 'race_group2', 'race_group3', 'race_group4']

# === Pairwise EOD comparisons on test set ===
records = []

for group in demographic_groups:
    if group not in test_data.columns:
        continue

    group_values = test_data[group].dropna().unique()

    for g1, g2 in combinations(group_values, 2):
        pair_df = test_data[test_data[group].isin([g1, g2])]
        if len(pair_df) < 20:
            continue  # skip small pair

        y_true = pair_df['case_status']
        y_pred = pair_df['prediction']
        sensitive_pair = pair_df[group]

        # Compute bootstrapped EOD CI
        eod_low, eod_high = bootstrap_pairwise_eod(y_true, y_pred, sensitive_pair)

        records.append({
            'Group': group,
            'Subgroup 1': g1,
            'Subgroup 2': g2,
            'EOD (95% CI)': f"{eod_low:.3f}–{eod_high:.3f}",
            'Sample Size': len(pair_df),
            'Flag (EOD > 0.1)': '⚠️' if eod_low > 0.1 else ''
        })

# === Print nicely ===
df_pairwise = pd.DataFrame(records)
print("\n=== Test Set Pairwise Equalized Odds Report ===")
print(tabulate(df_pairwise, headers="keys", tablefmt="fancy_grid", showindex=False))


## Check Equalized odds Violations -  group

https://fairlearn.org/v0.7.0/api_reference/fairlearn.metrics.html

fairlearn.metrics.equalized_odds_difference(y_true, y_pred, *, sensitive_features, method='between_groups', sample_weight=None)[source]
Calculate the equalized odds difference.

The greater of two metrics: true_positive_rate_difference and false_positive_rate_difference. The former is the difference between the largest and smallest of
, across all values
 of the sensitive feature(s). The latter is defined similarly, but for
. The equalized odds difference of 0 means that all groups have the same true positive, true negative, false positive, and false negative rates.

In [ ]:
import pandas as pd
import numpy as np
from fairlearn.metrics import equalized_odds_difference

# === Age binning ===
def bin_age(df):
    bin_edges = [0, 70, 80, 90, float('inf')]
    labels = ['<70', '70-80', '80-90', '>90']
    df['age_group'] = pd.cut(df['age'], bins=bin_edges, labels=labels, right=False)
    return df

# === Bootstrap EOD CI only ===
def bootstrap_eod(y_true, y_pred, sensitive_features, n_iterations=100):
    eods = []
    data = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred, 'sensitive': sensitive_features})

    for _ in range(n_iterations):
        sample = data.sample(frac=1.0, replace=True)
        try:
            eod = equalized_odds_difference(sample['y_true'], sample['y_pred'], sensitive_features=sample['sensitive'])
            eods.append(eod)
        except:
            continue

    if len(eods) == 0:
        return (np.nan, np.nan)

    return (
        np.nanpercentile(eods, 2.5),
        np.nanpercentile(eods, 97.5)
    )

# === Apply age binning ===
final_df = bin_age(final_df)

# === Split data ===
test_data = final_df[final_df['split'] == 'test']
train_data = final_df[final_df['split'] == 'train']

# === Demographic groups ===
demographic_groups = ['age_group', 'gender', 'insurance_group', 'language_group',
                      'race_group1', 'race_group2', 'race_group3', 'race_group4']

# === Run analysis for Train and Test ===
for data_split_name, data in zip(['Train', 'Test'], [train_data, test_data]):
    records = []

    for group in demographic_groups:
        if group not in data.columns:
            continue

        sensitive = data[group]
        valid = sensitive.notna()
        y_true = data.loc[valid, 'case_status']
        y_pred = data.loc[valid, 'prediction']
        sensitive_group = sensitive[valid]

        if len(sensitive_group.unique()) < 2:
            continue  # skip if only one group value

        # Compute EOD CI
        eod_low, eod_high = bootstrap_eod(y_true, y_pred, sensitive_group)

        records.append({
            'Group': group,
            'Equalized Odds Difference (95% CI)': f"{eod_low:.3f}–{eod_high:.3f}",
            'Sample Size': len(sensitive_group),
            'Flag (EOD > 0.1)': '⚠️' if eod_low > 0.1 else ''
        })

    # Create and display results
    df_metrics = pd.DataFrame(records)
    df_metrics.set_index('Group', inplace=True)

    print(f"\n=== {data_split_name} Equalized Odds Report ===")
    print(df_metrics)
